<a href="https://colab.research.google.com/github/tobias-octon-burke/googlecolab/blob/main/strava_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Strava test

use the strava API to extract rowing data

In [3]:
from google.colab import userdata
client_id = userdata.get('client_id')
client_secret = userdata.get('client_secret')
token = userdata.get('token')

# create the required url for creating a token for requests
url = f'http://www.strava.com/oauth/authorize?client_id={client_id}&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all'
url

'http://www.strava.com/oauth/authorize?client_id=119406&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=profile:read_all,activity:read_all'

In [4]:
import requests
import json
# Make Strava auth API call with your
# client_code, client_secret and code
response = requests.post(
                    url = 'https://www.strava.com/oauth/token',
                    data = {
                            'client_id': client_id,
                            'client_secret': client_secret,
                            'code': token,
                            'grant_type': 'authorization_code'
                            }
                )
#Save json response as a variable
strava_tokens = response.json()
# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)
# Open JSON file and print the file contents
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)


{'token_type': 'Bearer', 'expires_at': 1704590059, 'expires_in': 21600, 'refresh_token': 'b5b315d8c5ff51c7ab9237ff8f971ea90ef4c0cc', 'access_token': '8ee3c9edddad2f0981da71292493802c1a26de88', 'athlete': {'id': 23765072, 'username': None, 'resource_state': 2, 'firstname': 'Toby', 'lastname': 'octon', 'bio': None, 'city': None, 'state': None, 'country': None, 'sex': 'M', 'premium': False, 'summit': False, 'created_at': '2017-07-28T21:08:41Z', 'updated_at': '2024-01-06T13:32:09Z', 'badge_type_id': 0, 'weight': 80.0, 'profile_medium': 'https://graph.facebook.com/10159374375370227/picture?height=256&width=256', 'profile': 'https://graph.facebook.com/10159374375370227/picture?height=256&width=256', 'friend': None, 'follower': None}}


In [6]:
import requests
from pandas import json_normalize
import json
import csv
# Get the tokens from file to connect to Strava
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)
# Loop through all activities
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']
# Get first page of activities from Strava with all fields
r = requests.get(url + '?access_token=' + access_token)
r = r.json()

df = json_normalize(r)
df.to_csv('strava_activities_all_fields.csv')

In [8]:
df.loc[df['sport_type']=='VirtualRow'].head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,...,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,workout_type,elev_high,elev_low,upload_id_str
0,2,Morning Virtual Row,8000.0,2100,2100,0.0,Rowing,VirtualRow,10503626365,2024-01-06T10:31:44Z,...,False,23765072,1,a10503626365,,2,NaN,NaN,NaN,NaN
1,2,Morning Virtual Row,7000.0,1828,1828,0.0,Rowing,VirtualRow,10497545659,2024-01-05T10:27:16Z,...,False,23765072,1,a10497545659,,2,NaN,NaN,NaN,NaN
2,2,Morning Virtual Row,6800.0,1800,1800,0.0,Rowing,VirtualRow,10484626955,2024-01-03T10:50:35Z,...,False,23765072,1,a10484626955,,2,NaN,NaN,NaN,NaN
3,2,Morning Virtual Row,6700.0,1800,1800,0.0,Rowing,VirtualRow,10478569908,2024-01-02T11:47:54Z,...,False,23765072,1,a10478569908,,2,NaN,NaN,NaN,NaN
